In [159]:
import tensorflow as tf
import pandas as pd
import sagemaker
tf.__version__

'2.1.0'

In [160]:
#Reading in data in folder
df = pd.read_csv('Iris.csv')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [161]:
len(df)

150

In [162]:
#Splitting data in 80-20 split to use testing data for model inference later
train = df.iloc[:120,:]
test = df.iloc[121:,:]
train = train.drop('Id', axis=1)
test = test.drop('Id', axis=1)

In [164]:
#Train and test csv
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [165]:
#Create a sagemaker session to be able to upload data to s3
import boto3
sagemaker_session = sagemaker.Session()

In [166]:
#Uploading data to S3 bucket titled "tf-iris-data"
prefix = "tf-iris-data"
training_input_path = sagemaker_session.upload_data('train.csv', key_prefix=prefix + '/training')

In [167]:
#this is the path we will use to train our model
training_input_path

's3://sagemaker-us-east-1-906815961619/tf-iris-data/training/train.csv'

In [168]:
#ensure training data has uploaded properly
training_data = pd.read_csv(training_input_path, sep = ',')
training_data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [169]:
#Need tf version and role for the Tensorflow Estimator
tf_version = tf.__version__
tf_version

'2.1.0'

In [170]:
#Sagemaker role, make sure you've allowed access to any S3 bucket
role = sagemaker.get_execution_role()
role

'arn:aws:iam::906815961619:role/service-role/AmazonSageMaker-ExecutionRole-20201209T165545'

In [171]:
#Use a tensorflow estimator from sagemaker to train model
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='train.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.c5.18xlarge',
                          framework_version=tf_version, 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 30
                          }
                         )

In [173]:
#Training
tf_estimator.fit({'train': training_input_path})

2020-12-10 00:53:59 Starting - Starting the training job...
2020-12-10 00:54:22 Starting - Launching requested ML instancesProfilerReport-1607561639: InProgress
......
2020-12-10 00:55:23 Starting - Preparing the instances for training......
2020-12-10 00:56:24 Downloading - Downloading input data
2020-12-10 00:56:24 Training - Downloading the training image..2020-12-10 00:56:41,062 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-12-10 00:56:41,069 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-12-10 00:56:41,574 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-12-10 00:56:41,591 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-12-10 00:56:41,606 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-12-10 00:56:41,615 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_pa

In [177]:
#Creating endpoint and deploying model
import time
tf_endpoint_name = 'tf-iris-model'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
tf_predictor = tf_estimator.deploy(initial_instance_count=1,instance_type='ml.m5.4xlarge',
                                   endpoint_name=tf_endpoint_name)
print("Success!")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!Success!


In [204]:
#Example inference
#Create an inference function to return what iris flower species class our model is predicting
#There are three classes in order: Iris-setosa, Iris-versicolor, Iris-virginica

#Sample test
predictions = tf_predictor.predict([5.1,3.5,1.4,0.2])['predictions'] #returns a list of lists

def inferenceFunction(preds):
    preds = max(preds)
    maxValue = max(preds)
    maxIndex = preds.index(maxValue)
    if maxIndex == 0:
        print("Iris-setosa")
    elif maxIndex == 1:
        print("Iris-versicolor")
    else:
        print("Iris-virginica")

results = inferenceFunction(predictions)
print(results)

Iris-setosa
None


In [205]:
#Cross check this result with the first column in dataframe with same input values
df.head(1)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
